In [1]:
using ITensors, ITensorMPS
using LinearAlgebra
using Plots


In [2]:
N = 10
s = siteinds("Qubit",N)

10-element Vector{Index{Int64}}:
 (dim=2|id=397|"Qubit,Site,n=1")
 (dim=2|id=714|"Qubit,Site,n=2")
 (dim=2|id=474|"Qubit,Site,n=3")
 (dim=2|id=909|"Qubit,Site,n=4")
 (dim=2|id=921|"Qubit,Site,n=5")
 (dim=2|id=331|"Qubit,Site,n=6")
 (dim=2|id=601|"Qubit,Site,n=7")
 (dim=2|id=698|"Qubit,Site,n=8")
 (dim=2|id=204|"Qubit,Site,n=9")
 (dim=2|id=87|"Qubit,Site,n=10")

In [3]:
function generate_basis_states(n)
    basis_states = []
    for i in 0:(2^n - 1)
        binary_str = bitstring(i)[end-n+1:end]  # Get last n bits of bitstring
        push!(basis_states, collect(binary_str))  # Collect splits string into array of characters
    end
    return basis_states
end

MPS_states = []
basis_states = generate_basis_states(N)
for i in 1:(2^N)
    push!(MPS_states, MPS(s,string.(basis_states[i])))
end
println(basis_states)

Any[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '1'], ['0', '0', '0', '0', '0', '0', '0', '0', '1', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '1', '1'], ['0', '0', '0', '0', '0', '0', '0', '1', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '1', '0', '1'], ['0', '0', '0', '0', '0', '0', '0', '1', '1', '0'], ['0', '0', '0', '0', '0', '0', '0', '1', '1', '1'], ['0', '0', '0', '0', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '0', '1'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '1'], ['0', '0', '0', '0', '0', '0', '1', '1', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '1', '0', '1'], ['0', '0', '0', '0', '0', '0', '1', '1', '1', '0'], ['0', '0', '0', '0', '0', '0', '1', '1', '1', '1'], ['0', '0', '0', '0', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '1', '0', '0', '0', '1'], ['0', '0', '0', '0', '0', '1', '0', '0', '1', '0'], ['0', '0

In [ ]:
# Taking the input 

# #input_array = [2,1,0,1,0,9,0,1,2,1,0,1,0,9,0,1,7,1,0,1,0,6,0,1,2,1,0,1,0,0,0,1,2,1,5,1,0,3,0,1,2,1,0,1,0,0,0,1,2,1,0,1,0,7,0,1,2,1,33,1,0,32,0,1,2,1,0,1,0,0,0,1,2,1,0,1,0,0,0,1,2,1,21,1,0,0,0,1,2,1,0,1,0,0,0,1,2,1,0,1,0,0,0,1,2,1,0,1,0,0,0,1,2,1,0,1,0,0,0,1,2,1,0,1,0,0,0,1]

# # Normalize the array

x = range(0, stop=2π, length=2^N)
input_array = cos.(x)
    
# input_array = rand(2^N)
 
array = input_array / norm(input_array) # Input

ψ = array[1] *MPS(s, string.(basis_states[1]))
for i in 2:2^N
    
    ψ += array[i] * MPS(s, string.(basis_states[i]))
end

println(maxlinkdim(ψ))

In [ ]:
println("Time taken: ", end_time - start_time, " seconds")

In [ ]:
#Define the Gates 
#gates = ITensor[]

start_time = time()

function H_gate(x)
    Hadamard = op("H",s[x]) # op  is  a function which converts matrices to ITensors
    push!(gates,Hadamard)
end

function swap_gate(i,j)
    swap = [ 1 0 0 0 ; 0 0 1 0; 0 1 0 0; 0 0 0 1] 
    swap_gate = op(swap, s[i] , s[j])
    push!(gates,swap_gate)
end

function phase_gate(x,i,j)
    phase = [ 1 0 0 0 ; 0 1 0 0; 0 0 1 0; 0 0 0 exp(-1* im * 2*π/2^x)] 
    phase_gate = op(phase, s[i] , s[j])
    # s[i] and s[j] are the control qubit 
    push!(gates,phase_gate)
end


In [ ]:
gates = ITensor[]

for i in 1:N
        
    H_gate(i)
        
    for j in i+1:N
        phase_gate(j,j-1,j)
        if j == N
            for k=N-1:2
                swap_gate(k-1,k)
            end
        else
            swap_gate(j-1,j)

        end
        
    end

    TEBD_cutoff = 1E-18
    ψ = apply(gates,ψ,cutoff=TEBD_cutoff)
    println("Maximum bond dimension in the MPS : " , maxlinkdim(ψ))
    empty!(gates)

end


In [ ]:
gates = ITensor[]

r_max = N

for i in 1:N
        
    H_gate(1)
        
    for j in 2:r_max
        phase_gate(j,j-1,j)
        swap_gate(j-1,j)
    end

    TEBD_cutoff = 1E-18
    ψ = apply(gates,ψ,cutoff=TEBD_cutoff)
    println("Maximum bond dimension in the MPS : " , maxlinkdim(ψ))
    empty!(gates)

    r_max = r_max - 1
end


In [ ]:
#Calculating innter products

inner_products = []
for i in 1:(2^N)
    push!(inner_products, inner(MPS_states[i], ψ))
    print(inner(MPS_states[i], ψ)*norm(input_array)*2^(N/2))
    println()
end
end_time = time()


In [95]:
println("Time taken: ", end_time - start_time, " seconds")

Time taken: 40.786056995391846 seconds


In [96]:
using FFTW
start_time = time()
fft(input_array)
end_time = time()


1.72708884653519e9

In [97]:
println("Time taken: ", end_time - start_time, " seconds")

Time taken: 0.0007882118225097656 seconds
